In [1]:
import pandas as pd

/home/codespace/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/codespace/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


In [2]:
df=pd.read_parquet('./yellow_tripdata_2023-01.parquet')

In [3]:
print("Total Columns:",len(df.columns))

Total Columns: 19


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3066766 entries, 0 to 3066765
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[us]
 2   tpep_dropoff_datetime  datetime64[us]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee           

In [5]:
df['duration'] = df.tpep_dropoff_datetime  - df.tpep_pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

In [6]:
print("Standard Deviation", df.duration.std())

Standard Deviation 42.59435124195458


In [7]:
df_cleaned = df[(df.duration >= 1) & (df.duration <= 60)]
perc = len(df_cleaned) / len(df) * 100
print(f"Percentage of records left after outlier removal: {perc:.2f}%")

Percentage of records left after outlier removal: 98.12%


In [8]:
from sklearn.feature_extraction import DictVectorizer

data_dicts = df_cleaned[['PULocationID', 'DOLocationID']].astype(str).to_dict(orient='records')

dv = DictVectorizer()
X_train  = dv.fit_transform(data_dicts)

print("Feature matrix shape:", X_train.shape)

Feature matrix shape: (3009173, 515)


In [9]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

target = 'duration'
y_train = df_cleaned[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

rmse = np.sqrt(mean_squared_error(y_train, y_pred))
print("Root Mean Squared Error:", rmse)

Root Mean Squared Error: 7.649261932106969


In [ ]:
def transform_data(path):
    df=pd.read_parquet(path)
    df['duration'] = df.tpep_dropoff_datetime  - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
    df_cleaned = df[(df.duration >= 1) & (df.duration <= 60)]
    data_dicts = df_cleaned[['PULocationID', 'DOLocationID']].astype(str).to_dict(orient='records')
    dv = DictVectorizer()
    X = dv.transform(data_dicts)
    y = df_cleaned[target].values
    return X, y

: 

In [ ]:
X_val,y_val = transform_data('./yellow_tripdata_2023-02.parquet')


y_pred = lr.predict(X_val)
rmse = np.sqrt(mean_squared_error(y_val, y_pred))
print("Validation Root Mean Squared Error:", rmse)